In [1]:
import ppt_top_brand as ppt_tb
import olah_data_top_brand as olah_tb

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np
import seaborn as sb
import openpyxl

cm = sb.light_palette("green", as_cmap=True)

# Data

In [3]:
df_input_clean = pd.read_excel('data input clean top brand.xlsx')
df_tracking = pd.read_excel('gabungan data tracking na.xlsx')
df_semuaElemen = pd.read_excel('gabungan fase.xlsx')
df_tambahan = pd.read_excel('data tambahan edit.xlsx')
df_nontambahan = pd.read_excel('data non tambahan.xlsx')
mascod_bedding = pd.read_excel('mascod bedding 2021.xlsx')
excel_tambahan = pd.ExcelFile('mascod tambahan.xlsx')
excel_non_tambahan = pd.ExcelFile('mascod non tambahan.xlsx')

code_top_brand_lainnya = pd.read_excel('urutan brand.xlsx')
code_top_brand = code_top_brand_lainnya.loc[:len(code_top_brand_lainnya)-2].sort_values(by='Sorting').reset_index(drop=True)

top10_brand = code_top_brand_lainnya[~(code_top_brand_lainnya['Brand']=='Lainnya') & ~(code_top_brand_lainnya['Brand']=='Total')]['Brand'].to_list()
sorter_brand = code_top_brand_lainnya['Sorting'][:len(code_top_brand_lainnya)]

In [4]:
client = 'Comforta'
subkategori = ['Bedding']

## Pendahuluan

In [5]:
sample_size = olah_tb.sample_size(df_input_clean, ['Kota','Random','Booster','Total'])

In [6]:
# sample_size

## Ringkasan Eksekutif

In [7]:
data_tracking, merek_tracking, tahun_ini = olah_tb.buat_data_tracking(df_tracking, subkategori)
subkategori = [subkategori[0].upper()]
df_semuaElemen, semua_elemen_sekarang, semua_elemen_kemarin, merek_top6 = olah_tb.top6_semuaElemen_DuaTahun(df_semuaElemen, subkategori, tahun_ini)

dt_client = df_semuaElemen[(df_semuaElemen.Brand.isin([client]))]
dt_client_5thn = dt_client[~(tahun_ini-dt_client['Tahun'] >= 5)]

In [8]:
dt_tracking = data_tracking[['Tahun','Merek','Top Brand Index']].pivot(index='Tahun',columns='Merek',
                                                                            values='Top Brand Index')
dt_tracking = dt_tracking.reset_index()
dt_tracking.columns.name = None

## profil responden

In [9]:
# Memberikan label nama brand ke data dan membuat data ir ya
df_laporan = olah_tb.data_laporan(df_input_clean, mascod_bedding,  top10_brand)
data_ir = df_laporan[df_laporan.IR=='Ya']
dt_nonIR = df_laporan[df_laporan.IR == 'Tidak']

In [10]:
profil_ir, profil_ir_persen = olah_tb.data_ir(df_laporan)

total_Usiar, tabelUsiar = olah_tb.profil_responden(data_ir, 'LU', 'usiar', sorter_brand)
total_Usia, tabelUsia = olah_tb.profil_responden(data_ir, 'LU', 'usia', sorter_brand)
mean_usia = [round(np.mean(data_ir.usia), 2)]
total_Pekerjaan, tabelPekerjaan = olah_tb.profil_responden(data_ir, 'LU', 'kerja', sorter_brand)
presentase_pekerjaan = olah_tb.presentase_profil(tabelPekerjaan)
total_Pendidikan, tabelPendidikan = olah_tb.profil_responden(data_ir, 'LU', 'didik', sorter_brand, olah_tb.pendidikan_urut())
total_Expandr, tabelExpandr = olah_tb.profil_responden(data_ir, 'LU', 'expandr', sorter_brand, olah_tb.ses_urut())

In [11]:
# tabelPendidikan

## Peta top brand

In [12]:
# Data untuk TOM, LU, Fl - Slide 35, 39, 48, 53
## Menghitung indeks
mind_share, total_bobot_tom = olah_tb.hitung_nilai('TOM', df_laporan, sorter_brand)
mind_share_nonuser, _ = olah_tb.hitung_nilai('TOM', df_laporan, sorter_brand, non_user = True)
market_share, _ = olah_tb.hitung_nilai('LU', data_ir, sorter_brand)
commitment_share, _ = olah_tb.hitung_nilai('FI', data_ir, sorter_brand)

## Menghitung gap
gap_mind_share = olah_tb.hitung_gap(mind_share, top10_brand)
gap_mind_share_nonuser = olah_tb.hitung_gap(mind_share_nonuser, top10_brand)
gap_market_share = olah_tb.hitung_gap(market_share, top10_brand)
gap_commitment_share = olah_tb.hitung_gap(commitment_share, top10_brand)

In [13]:
# Data untuk TOM, LU, Fl dengan breakdown
## Mind share - Slide 36 s/d 38
mind_share_kota = olah_tb.hitung_nilai_crosstab(kriteria = 'TOM', by = 'kota', data_used = df_laporan, 
                      indeks_brand = sorter_brand, indeks_kolom = olah_tb.kota_urut())
mind_share_expandr = olah_tb.hitung_nilai_crosstab(kriteria = 'TOM', by = 'expandr', data_used = df_laporan, 
                      indeks_brand = sorter_brand, indeks_kolom = olah_tb.ses_urut())
mind_share_usiar = olah_tb.hitung_nilai_crosstab(kriteria = 'TOM', by = 'usiar', data_used = df_laporan, 
                      indeks_brand = sorter_brand)
mind_share_sex = olah_tb.hitung_nilai_crosstab(kriteria = 'TOM', by = 'sex', data_used = df_laporan, 
                      indeks_brand = sorter_brand)
mind_share_kota = olah_tb.fungsi_by('TOM','kota',mind_share_kota,df_laporan)
mind_share_expandr = olah_tb.fungsi_by('TOM','expandr',mind_share_expandr,df_laporan)
mind_share_usiar = olah_tb.fungsi_by('TOM','usiar',mind_share_usiar,df_laporan)
mind_share_sex = olah_tb.fungsi_by('TOM','sex',mind_share_sex,df_laporan)


## Mind share non user - Slide 40 s/d 42
mind_share_kota_nonuser = olah_tb.hitung_nilai_crosstab(kriteria = 'TOM', by = 'kota', data_used = df_laporan, 
                      indeks_brand = sorter_brand, indeks_kolom = olah_tb.kota_urut(), non_user=True)
mind_share_expandr_nonuser = olah_tb.hitung_nilai_crosstab(kriteria = 'TOM', by = 'expandr', data_used = df_laporan, 
                      indeks_brand = sorter_brand, indeks_kolom = olah_tb.ses_urut(), non_user=True)
mind_share_usiar_nonuser = olah_tb.hitung_nilai_crosstab(kriteria = 'TOM', by = 'usiar', data_used = df_laporan, 
                      indeks_brand = sorter_brand, non_user=True)
mind_share_sex_nonuser = olah_tb.hitung_nilai_crosstab(kriteria = 'TOM', by = 'sex', data_used = df_laporan, 
                      indeks_brand = sorter_brand, non_user=True)
mind_share_kota_nonuser = olah_tb.fungsi_by('TOM','kota',mind_share_kota_nonuser,dt_nonIR)
mind_share_expandr_nonuser = olah_tb.fungsi_by('TOM','expandr',mind_share_expandr_nonuser,dt_nonIR)
mind_share_usiar_nonuser = olah_tb.fungsi_by('TOM','usiar',mind_share_usiar_nonuser,dt_nonIR)
mind_share_sex_nonuser = olah_tb.fungsi_by('TOM','sex',mind_share_sex_nonuser,dt_nonIR)


## Market share - Slide 48 s/d 51
market_share_kota = olah_tb.hitung_nilai_crosstab(kriteria = 'LU', by = 'kota', data_used = data_ir, 
                      indeks_brand = sorter_brand, indeks_kolom = olah_tb.kota_urut())
market_share_expandr = olah_tb.hitung_nilai_crosstab(kriteria = 'LU', by = 'expandr', data_used = data_ir, 
                      indeks_brand = sorter_brand, indeks_kolom = olah_tb.ses_urut())
market_share_usiar = olah_tb.hitung_nilai_crosstab(kriteria = 'LU', by = 'usiar', data_used = data_ir, 
                      indeks_brand = sorter_brand)
market_share_sex = olah_tb.hitung_nilai_crosstab(kriteria = 'LU', by = 'sex', data_used = data_ir, 
                      indeks_brand = sorter_brand)
market_share_kota = olah_tb.fungsi_by('LU','kota',market_share_kota,data_ir)
market_share_expandr = olah_tb.fungsi_by('LU','expandr',market_share_expandr,data_ir)
market_share_usiar = olah_tb.fungsi_by('LU','usiar',market_share_usiar,data_ir)
market_share_sex = olah_tb.fungsi_by('LU','sex',market_share_sex,data_ir)


## Commitment share - Slide 54 s/d 56
commitment_share_kota = olah_tb.hitung_nilai_crosstab(kriteria = 'FI', by = 'kota', data_used = data_ir, 
                      indeks_brand = sorter_brand, indeks_kolom = olah_tb.kota_urut())
commitment_share_expandr = olah_tb.hitung_nilai_crosstab(kriteria = 'FI', by = 'expandr', data_used = data_ir, 
                      indeks_brand = sorter_brand, indeks_kolom = olah_tb.ses_urut())
commitment_share_usiar = olah_tb.hitung_nilai_crosstab(kriteria = 'FI', by = 'usiar', data_used = data_ir, 
                      indeks_brand = sorter_brand)
commitment_share_sex = olah_tb.hitung_nilai_crosstab(kriteria = 'FI', by = 'sex', data_used = data_ir, 
                      indeks_brand = sorter_brand)
commitment_share_kota = olah_tb.fungsi_by('LU','kota',commitment_share_kota,data_ir)
commitment_share_expandr = olah_tb.fungsi_by('LU','expandr',commitment_share_expandr,data_ir)
commitment_share_usiar = olah_tb.fungsi_by('LU','usiar',commitment_share_usiar,data_ir)
commitment_share_sex = olah_tb.fungsi_by('LU','sex',commitment_share_sex,data_ir)

In [14]:
# Data untuk grafik tbi
## TBI - Slide 30
dt_tbi = olah_tb.hitung_tbi(mind_share, market_share, commitment_share)
gap_tbi = olah_tb.hitung_gap(dt_tbi, top10_brand)

## TBI dengan Breakdown - Slide 31 s/d 33
dt_tbi_kota = olah_tb.crosstab_tbi(mind_share_kota.data, market_share_kota.data, commitment_share_kota.data)
dt_tbi_expandr = olah_tb.crosstab_tbi(mind_share_expandr.data, market_share_expandr.data, commitment_share_expandr.data)
dt_tbi_usiar = olah_tb.crosstab_tbi(mind_share_usiar.data, market_share_usiar.data, commitment_share_usiar.data)
dt_tbi_sex = olah_tb.crosstab_tbi(mind_share_sex.data, market_share_sex.data, commitment_share_sex.data)

indeks_tbi_kota = olah_tb.conditional_formating(dt_tbi_kota)
indeks_tbi_expandr = olah_tb.conditional_formating(dt_tbi_expandr)
indeks_tbi_usiar = olah_tb.conditional_formating(dt_tbi_usiar)
indeks_tbi_sex = olah_tb.conditional_formating(dt_tbi_sex)

In [15]:
# Data untuk grafik Unaided 
## Data Unaided- Slide 43
df_unaided = olah_tb.get_dataframe_unaided(df_laporan)

dt_unaided = olah_tb.hitung_unaided(df_unaided, total_bobot_tom, sorter_brand)
gap_unaided = olah_tb.hitung_gap(dt_unaided, top10_brand)

dt_selisih = dt_unaided.copy()
dt_selisih['Unaided'] = dt_unaided['Unaided']-mind_share['Mind Share']
dt_selisih.columns = ['Brand','Selisih UN-TOM']

## Data unaided dengan breakdown - Slide 44 s/d 46
unaided_kota = olah_tb.hitung_nilai_crosstab(kriteria = 'UN', by = 'kota', data_used = df_unaided, 
                                     indeks_brand = sorter_brand, indeks_kolom = olah_tb.kota_urut(), dt_tom=df_laporan)
unaided_usiar = olah_tb.hitung_nilai_crosstab(kriteria = 'UN', by = 'usiar', data_used = df_unaided, 
                                     indeks_brand = sorter_brand, dt_tom=df_laporan)
unaided_expandr = olah_tb.hitung_nilai_crosstab(kriteria = 'UN', by = 'expandr', data_used = df_unaided, 
                                     indeks_brand = sorter_brand, indeks_kolom = olah_tb.ses_urut(), dt_tom=df_laporan)
unaided_sex = olah_tb.hitung_nilai_crosstab(kriteria = 'UN', by = 'sex', data_used = df_unaided, 
                                     indeks_brand = sorter_brand, dt_tom=df_laporan)
unaided_kota = olah_tb.fungsi_by('TOM','kota',unaided_kota,df_laporan)
unaided_expandr = olah_tb.fungsi_by('TOM','expandr',unaided_expandr,df_laporan)
unaided_usiar = olah_tb.fungsi_by('TOM','usiar',unaided_usiar,df_laporan)
unaided_sex = olah_tb.fungsi_by('TOM','sex',unaided_sex,df_laporan)

In [16]:
mind_share_nonuser.columns =['Brand','Mind Share (non-user)']
gap_mind_share.columns = ['Brand','Gap Mind Share']
gap_mind_share_nonuser.columns = ['Brand', 'Gap Mind Share (non-user)']
gap_market_share.columns = ['Brand', 'Gap Market Share']
gap_commitment_share.columns = ['Brand','Gap Commitment Share']
gap_tbi.columns = ['Brand','Gap TBI']
gap_unaided.columns = ['Brand','Gap Unaided']

tabel_indeks = pd.concat([dt_tbi.set_index('Brand'),mind_share.set_index('Brand'), mind_share_nonuser.set_index('Brand'),
                         dt_unaided.set_index('Brand'),market_share.set_index('Brand'), commitment_share.set_index('Brand')], 
                         axis=1)
tabel_indeks = tabel_indeks.reset_index()
tabel_gap = pd.concat([gap_tbi.set_index('Brand'),gap_mind_share.set_index('Brand'), gap_mind_share_nonuser.set_index('Brand'),
                       gap_unaided.set_index('Brand'), dt_selisih.set_index('Brand'),
                       gap_market_share.set_index('Brand'), gap_commitment_share.set_index('Brand')], axis=1)
tabel_gap = tabel_gap.reset_index()

## Brand Switching

In [17]:
# Data untuk grafik Brand Switching
## Data grafik - Slide 59
brand_switch = olah_tb.hitung_brand_switch(df_laporan, code_top_brand_lainnya, sorter_brand)
n_sample = olah_tb.n_sample_user(data_ir, code_top_brand_lainnya)

## Data tabel FI-LU - Slide 60
tabel_brand_switching = olah_tb.tabel_brand_switch(df_laporan, code_top_brand, n_sample)

## Data Convertion Rate - SLide 62
conv_rate = olah_tb.hitung_conv_rate(data_used = df_laporan, list_sorted_brand = code_top_brand)

In [18]:
brand_switching_analysis = pd.concat([brand_switch.set_index('Brand'), conv_rate.set_index('Brand')], 
                                     keys=['Brand Switching','Conversion Rate'], axis=1)

## Brand diagnostic & Competitor Landscape

In [19]:
# Data untuk tabel dan grafik Brand Diagnostic - Slide 66 dan 67
dt_diagnostic = olah_tb.brand_diagnostic(mind_share, market_share, commitment_share)

In [20]:
# Data untuk tabel Competitor Landscape - Slide 71 s/d 77
kt_ = olah_tb.kota_urut()

bobot_cl, competitor_all, competitor_landscape = olah_tb.hitung_bobot_cl(data_ir, code_top_brand)
cl_overall = olah_tb.hitung_competitor_landscape('Overall',competitor_all, bobot_cl)
cl_perkota = [olah_tb.hitung_competitor_landscape(kt_[i], competitor_landscape[i],bobot_cl) for i in range(len(competitor_landscape))]

In [21]:
cl_overall_ = olah_tb.conditional_formating(cl_overall, 'competitor')
cl_perkota_ = []*len(cl_perkota)
cl_perkota_ = [olah_tb.conditional_formating(x_, 'competitor') for x_ in cl_perkota]

## General Information

### Media Habit

In [22]:
# Data untuk grafik Media Habit - Slide 80 s/d 85
mascod_mhabit = pd.read_excel(excel_tambahan, sheet_name = 'p8')
tab_mhabit = olah_tb.hitung_tambahan(data_used=df_tambahan, kode_awal='p8', mascod = mascod_mhabit, top = 10)
tab_mhabit = tab_mhabit.reset_index()
tab_mhabit['Kriteria'] = tab_mhabit['Kriteria'].astype('str')

mascod_tv = pd.read_excel(excel_tambahan, sheet_name = 'tv')
tab_mhabit_tv = olah_tb.hitung_tambahan(data_used=df_tambahan, kode_awal='tv', mascod = mascod_tv, top = 10)
tab_mhabit_tv = tab_mhabit_tv.reset_index()
tab_mhabit_tv['Kriteria'] = tab_mhabit_tv['Kriteria'].astype('str')

mascod_acaratv = pd.read_excel(excel_tambahan, sheet_name = 'acaratv')
tab_mhabit_acaratv = olah_tb.hitung_tambahan(data_used=df_tambahan, kode_awal='acaratv', mascod = mascod_acaratv, top = 10)
tab_mhabit_acaratv = tab_mhabit_acaratv.reset_index()
tab_mhabit_acaratv['Kriteria'] = tab_mhabit_acaratv['Kriteria'].astype('str')

mascod_koran = pd.read_excel(excel_tambahan, sheet_name = 'koran')
tab_mhabit_koran = olah_tb.hitung_tambahan(data_used=df_tambahan, kode_awal='koran', mascod = mascod_koran, top = 10)
tab_mhabit_koran = tab_mhabit_koran.reset_index()
tab_mhabit_koran['Kriteria'] = tab_mhabit_koran['Kriteria'].astype('str')

mascod_majalah = pd.read_excel(excel_tambahan, sheet_name = 'majalah')
tab_mhabit_majalah = olah_tb.hitung_tambahan(data_used=df_tambahan, kode_awal='majalah', mascod = mascod_majalah, top = 10)
tab_mhabit_majalah = tab_mhabit_majalah.reset_index()
tab_mhabit_majalah['Kriteria'] = tab_mhabit_majalah['Kriteria'].astype('str')

mascod_tabloid = pd.read_excel(excel_tambahan, sheet_name = 'tabloid')
tab_mhabit_tabloid = olah_tb.hitung_tambahan(data_used=df_tambahan, kode_awal='tabloid', mascod = mascod_tabloid, top = 10)
tab_mhabit_tabloid = tab_mhabit_tabloid.reset_index()
tab_mhabit_tabloid['Kriteria'] = tab_mhabit_tabloid['Kriteria'].astype('str')

### Internet & Media Sosial

In [23]:
dt_olshop = olah_tb.filter_tambahan(df_tambahan, kode_awal = 'p18_1_0', incl_kota = True, kolom_kota = 'Kota_1_0')
mascod_kota = pd.read_excel(excel_tambahan, sheet_name = 'Kota_1_0')
mascod_olshop = pd.read_excel(excel_tambahan, sheet_name = 'p18_1_0')
profil_olshop, profil_olshop_persen = olah_tb.data_olshop(dt_olshop,  mascod_kota, mascod_olshop)

dt_pengeluaran = olah_tb.filter_tambahan(df_tambahan, kode_awal = 'p22_1', incl_kota = False)
dt_pengeluaran = dt_pengeluaran.dropna()
rata_pengeluaran, n_sampel_pengeluaran = np.mean(dt_pengeluaran), len(dt_pengeluaran)

mascod_jenis_barang = pd.read_excel(excel_tambahan, sheet_name = 'p20')
tab_ism_jenis_barang = olah_tb.hitung_tambahan(data_used=df_tambahan, kode_awal='p20_1', mascod = mascod_jenis_barang, top = 10)
tab_ism_jenis_barang = tab_ism_jenis_barang.reset_index()
tab_ism_jenis_barang['Kriteria'] = tab_ism_jenis_barang['Kriteria'].astype('str')

mascod_pembayaran = pd.read_excel(excel_tambahan, sheet_name = 'p23')
tab_ism_pembayaran = olah_tb.hitung_tambahan(data_used=df_tambahan, kode_awal='p23', mascod = mascod_pembayaran, top = 7)
tab_ism_pembayaran = tab_ism_pembayaran.reset_index()
tab_ism_pembayaran['Kriteria'] = tab_ism_pembayaran['Kriteria'].astype('str')

mascod_chat = pd.read_excel(excel_tambahan, sheet_name = 'p13')
tab_ism_chat = olah_tb.hitung_tambahan(data_used=df_tambahan, kode_awal='p13', mascod = mascod_chat, top = 5)
tab_ism_chat = tab_ism_chat.reset_index()
tab_ism_chat['Kriteria'] = tab_ism_chat['Kriteria'].astype('str')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [24]:
mascod_gadget_dimiliki = pd.read_excel(excel_tambahan, sheet_name = 'p9')
tab_ism_gadget_dimiliki = olah_tb.hitung_tambahan(data_used=df_tambahan, kode_awal='p9_1', mascod = mascod_gadget_dimiliki, top = 9)
tab_ism_gadget_sering = olah_tb.hitung_tambahan(data_used=df_tambahan, kode_awal='p10', mascod = mascod_gadget_dimiliki)
tab_ism_gadget_dimiliki.columns = ['Count_1','Presentase_1']
tab_ism_gadget_sering.columns = ['Count_2','Presentase_2']
tab_ism_gadget = pd.concat([tab_ism_gadget_dimiliki,tab_ism_gadget_sering], axis = 1)
tab_ism_gadget = tab_ism_gadget.fillna(0)
list_gadget = tab_ism_gadget.index.to_list()
list_gadget = [x for x in list_gadget if (x not in ['Multirespon','Tidak ada/ Tidak Punya'])]
list_gadget = list_gadget + ['Tidak ada/ Tidak Punya', 'Multirespon']
tab_ism_gadget = tab_ism_gadget.reindex(index = list_gadget)
tab_ism_gadget.index.rename('Kriteria', inplace = True)
tab_ism_gadget = tab_ism_gadget.reset_index()
tab_ism_gadget['Kriteria'] = tab_ism_gadget['Kriteria'].astype('str')

mascod_sosmed_dimiliki = pd.read_excel(excel_tambahan, sheet_name = 'p11')
tab_ism_sosmed_dimiliki = olah_tb.hitung_tambahan(data_used=df_tambahan, kode_awal='p11_1', mascod = mascod_sosmed_dimiliki, top = 3)
tab_ism_sosmed_sering = olah_tb.hitung_tambahan(data_used=df_tambahan, kode_awal='p12', mascod = mascod_sosmed_dimiliki)
tab_ism_sosmed_dimiliki.columns = ['Count_1','Presentase_1']
tab_ism_sosmed_sering.columns = ['Count_2','Presentase_2']
tab_ism_sosmed = pd.concat([tab_ism_sosmed_dimiliki,tab_ism_sosmed_sering], axis = 1)
tab_ism_sosmed = tab_ism_sosmed.fillna(0)
list_sosmed = tab_ism_sosmed.index.to_list()
list_sosmed = [x for x in list_sosmed if (x not in ['Multirespon','Tidak ada/ Tidak Punya'])]
list_sosmed = list_sosmed + ['Tidak ada/ Tidak Punya', 'Multirespon']
tab_ism_sosmed = tab_ism_sosmed.reindex(index = list_sosmed)
tab_ism_sosmed = tab_ism_sosmed.iloc[[0,1,2,-1],:]
tab_ism_sosmed.index.rename('Kriteria', inplace = True)
tab_ism_sosmed = tab_ism_sosmed.reset_index()
tab_ism_sosmed['Kriteria'] = tab_ism_sosmed['Kriteria'].astype('str')

In [25]:
mascod_akses = pd.read_excel(excel_non_tambahan, sheet_name = 'akses')
tab_ism_akses = olah_tb.hitung_tambahan(data_used=df_nontambahan, kode_awal='akses', mascod = mascod_akses, top = 10)
tab_ism_akses = tab_ism_akses.reset_index()
tab_ism_akses['Kriteria'] = tab_ism_akses['Kriteria'].astype('str')

mascod_aktivitas = pd.read_excel(excel_non_tambahan, sheet_name = 'aktivitas')
tab_ism_aktivitas = olah_tb.hitung_tambahan(data_used=df_nontambahan, kode_awal='aktivitas', mascod = mascod_aktivitas, top = 10)
tab_ism_aktivitas = tab_ism_aktivitas.reset_index()
tab_ism_aktivitas['Kriteria'] = tab_ism_aktivitas['Kriteria'].astype('str')

mascod_waktu = pd.read_excel(excel_non_tambahan, sheet_name = 'waktu')
tab_ism_waktu = olah_tb.hitung_tambahan(data_used=df_nontambahan, kode_awal='waktu', mascod = mascod_waktu, top = 10)
tab_ism_waktu = tab_ism_waktu.reset_index()
tab_ism_waktu['Kriteria'] = tab_ism_waktu['Kriteria'].astype('str')

mascod_media_upload = pd.read_excel(excel_non_tambahan, sheet_name = 'media_upload')
tab_ism_media_upload = olah_tb.hitung_tambahan(data_used=df_nontambahan, kode_awal='media_upload', mascod = mascod_media_upload, top = 10)
tab_ism_media_upload = tab_ism_media_upload.reset_index()
tab_ism_media_upload['Kriteria'] = tab_ism_media_upload['Kriteria'].astype('str')

mascod_media_download = pd.read_excel(excel_non_tambahan, sheet_name = 'media_download')
tab_ism_media_download = olah_tb.hitung_tambahan(data_used=df_nontambahan, kode_awal='media_download', mascod = mascod_media_download, top = 10)
tab_ism_media_download = tab_ism_media_download.reset_index()
tab_ism_media_download['Kriteria'] = tab_ism_media_download['Kriteria'].astype('str')

mascod_email = pd.read_excel(excel_non_tambahan, sheet_name = 'email')
tab_ism_email = olah_tb.hitung_tambahan(data_used=df_nontambahan, kode_awal='email', mascod = mascod_email, top = 10)
tab_ism_email = tab_ism_email.reset_index()
tab_ism_email['Kriteria'] = tab_ism_email['Kriteria'].astype('str')

mascod_game = pd.read_excel(excel_non_tambahan, sheet_name = 'game')
tab_ism_game = olah_tb.hitung_tambahan(data_used=df_nontambahan, kode_awal='game', mascod = mascod_game, top = 10)
tab_ism_game = tab_ism_game.reset_index()
tab_ism_game['Kriteria'] = tab_ism_game['Kriteria'].astype('str')

mascod_berita = pd.read_excel(excel_non_tambahan, sheet_name = 'berita')
tab_ism_berita = olah_tb.hitung_tambahan(data_used=df_nontambahan, kode_awal='berita', mascod = mascod_berita, top = 10)
tab_ism_berita = tab_ism_berita.reset_index()
tab_ism_berita['Kriteria'] = tab_ism_berita['Kriteria'].astype('str')

mascod_streaming = pd.read_excel(excel_non_tambahan, sheet_name = 'streaming')
tab_ism_streaming = olah_tb.hitung_tambahan(data_used=df_nontambahan, kode_awal='streaming', mascod = mascod_streaming, top = 10)
tab_ism_streaming = tab_ism_streaming.reset_index()
tab_ism_streaming['Kriteria'] = tab_ism_streaming['Kriteria'].astype('str')

mascod_olshop = pd.read_excel(excel_non_tambahan, sheet_name = 'olshop')
tab_ism_olshop = olah_tb.hitung_tambahan(data_used=df_nontambahan, kode_awal='olshop', mascod = mascod_olshop, top = 10)
tab_ism_olshop = tab_ism_olshop.reset_index()
tab_ism_olshop['Kriteria'] = tab_ism_olshop['Kriteria'].astype('str')

## Hitung n Sampel dan Multirespon

In [26]:
# Peta Top Brand
nSampel_mind_share = olah_tb.hitung_nSampel(df_laporan, 'TOM')

dt_nonIR = df_laporan[df_laporan.IR == 'Tidak']
nSampel_mind_share_nonUser = olah_tb.hitung_nSampel(dt_nonIR, 'TOM')

unaided_multirespon = [round(sum(dt_unaided['Unaided']), 1)]
unaided_nSampel = [nSampel_mind_share.iloc[-1,0]]
nSampel_mult_unaided = pd.DataFrame({'nSampel': unaided_nSampel, 'Total multirespon': unaided_multirespon})

nSampel_market_share = olah_tb.hitung_nSampel(df_laporan, 'LU')
nSampel_commitment_share = olah_tb.hitung_nSampel(df_laporan, 'FI')

## simpan

In [27]:
nama_output = "tabel output laporan top brand1 %s.xlsx"%(client)
list_simpan = [sample_size, dt_tracking,
               tabelUsiar, tabelUsia, tabelPendidikan, tabelPekerjaan, presentase_pekerjaan, tabelExpandr,
               mind_share_kota, mind_share_expandr, mind_share_usiar, mind_share_sex,
               mind_share_kota_nonuser, mind_share_expandr_nonuser, mind_share_usiar_nonuser, mind_share_sex_nonuser,
               market_share_kota, market_share_expandr, market_share_usiar, market_share_sex,
               commitment_share_kota, commitment_share_expandr, commitment_share_usiar, commitment_share_sex,
               indeks_tbi_kota, indeks_tbi_expandr, indeks_tbi_usiar, indeks_tbi_sex,
               unaided_kota, unaided_expandr, unaided_usiar, unaided_sex,
               tabel_indeks, tabel_gap, brand_switching_analysis, tabel_brand_switching, dt_diagnostic, 
              cl_overall_ , cl_perkota_]
list_nama_simpan = ['sample size', 'data tracking',
                    'sample Usiar','sample Usia','sample Pendidikan','sample Pekerjaan','presentase Pekerjaan','sample Expandr',
                    'mind_share_kota', 'mind_share_expandr', 'mind_share_usiar', 'mind_share_sex',
                    'mind_share_kota_nonuser', 'mind_share_expandr_nonuser', 'mind_share_usiar_nonuser', 'mind_share_sex_nonuser',
                    'market_share_kota', 'market_share_expandr', 'market_share_usiar', 'market_share_sex',
                    'commitment_share_kota', 'commitment_share_expandr', 'commitment_share_usiar', 'commitment_share_sex',                    
                    'indeks tbi kota','indeks tbi expandr', 'indeks tbi usiar','indeks tbi sex',
                    'unaided kota','unaided expandr','unaided usiar','unaided sex',
                    'tabel indeks','tabel gap', 'brand switching analysis', 'tabel brand switching', 'brand diagnostic',
                   'competitor landscape', 'competitor kota']
writer = pd.ExcelWriter(nama_output)

for i in range(len(list_simpan)):
    if ('presentase Pekerjaan' in list_nama_simpan[i]) or  ('indeks ' in list_nama_simpan[i]) or ('brand switching' in list_nama_simpan[i]) or ('share' in list_nama_simpan[i]) or ('unaided' in list_nama_simpan[i]) or ('competitor landscape'==list_nama_simpan[i]):
        list_simpan[i].to_excel(writer,sheet_name=list_nama_simpan[i])
    elif ('competitor kota' == list_nama_simpan[i]):
        for j in range(len(cl_perkota_)):
            list_simpan[i][j].to_excel(writer,sheet_name=list_nama_simpan[i]+cl_perkota_[j].data.index.names[0])
    else:
        list_simpan[i].to_excel(writer,sheet_name=list_nama_simpan[i], index=False)
writer.save()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\formats\excel.py:331: CSSWarning: Unhandled color format: 'royalblue'
  warnings.warn(f"Unhandled color format: {repr(val)}", CSSWarning)


# PPT

In [28]:
template_ppt = ppt_tb.FileTemplate('template laporan top brand 2021.pptx')
excel = pd.ExcelFile('tabel output laporan top brand1 Comforta.xlsx')

## Pendahuluan

In [29]:
tabel_sample_size = ppt_tb.buat_tabel(template_ppt = template_ppt, 
                                      nomor_slide=6, 
                                      baris=len(sample_size)+1, kolom=len(sample_size.columns)+1, 
                                      kiri=0.25, atas=1, lebar=4.25, tinggi=5)
tabel_sample_size.columns[0].width = ppt_tb.Inches(.5)
tabel_sample_size.columns[1].width = ppt_tb.Inches(1.2)
# tabel_sample_size.columns[2].width = ppt_tb.Inches(.75)
# tabel_sample_size.columns[3].width = ppt_tb.Inches(.75)
# tabel_sample_size.columns[4].width = ppt_tb.Inches(.75)

tabel_sample_size.rows[0].height = ppt_tb.Inches(.47)

for i in range(1, len(sample_size)-1):
    tabel_sample_size.rows[i].height = ppt_tb.Inches(.33)
tabel_sample_size.rows[len(sample_size)].height = ppt_tb.Inches(.47)

ppt_tb.plot_sample_size(sample_size, tabel_sample_size)

## Ringkasan Eksekutif

In [30]:
ppt_tb.plot_tracking(template_ppt, 13, data_tracking, merek_tracking)
ppt_tb.plot_kemarin_sekarang(template_ppt, 14, tahun_ini, semua_elemen_kemarin, semua_elemen_sekarang, merek_top6)
ppt_tb.plot_tracking_client(template_ppt, 15, dt_client)
ppt_tb.plot_line_tracking_client(template_ppt, 16, dt_client_5thn)

## Profil responden

In [31]:
ppt_tb.plot_data_ir_kota(template_ppt, 22, profil_ir_persen)
ppt_tb.pie_chart_ir(template_ppt, 22, profil_ir_persen, .5, 2, 4.25, 3.32)
ppt_tb.plot_nSampel_multirespon(template_ppt, 22, profil_ir, 'Lato Light', 10,
                            .5, 4.75, 1, .25, -1, -1, multirespon_ = False)

ppt_tb.donut_chart(template_ppt, 23, total_Usiar,  .25, 1, 4.5, 3.5)
ppt_tb.bar_chart_profil(template_ppt, 23, total_Pendidikan, 5, 1.25, 4, 3)
ppt_tb.bar_chart_profil(template_ppt, 23, total_Pekerjaan, .5, 4.5, 4, 2.5, 5)
ppt_tb.donut_chart(template_ppt, 23, total_Expandr,   5.5, 4.5, 4.5, 3.5, True)
ppt_tb.plot_text(template_ppt, 23,"Mean usia", mean_usia, 'Tahoma', 10, .5, 3.75, 1.5, .25)

ppt_tb.grafik_profil_responden(template_ppt, 24, tabelUsiar)
ppt_tb.plot_tabel_n_sampel(template_ppt, 24, tabelUsia, True)

ppt_tb.grafik_profil_responden(template_ppt, 25, tabelExpandr)
ppt_tb.plot_tabel_n_sampel(template_ppt, 25, tabelExpandr)

ppt_tb.grafik_profil_responden(template_ppt, 26, tabelPendidikan)
ppt_tb.plot_tabel_n_sampel(template_ppt, 26, tabelPendidikan)

ppt_tb.plot_tabel_pekerjaan(template_ppt, 27, excel, 'presentase Pekerjaan')

## Peta top brand

In [32]:
ppt_tb.ppt_plot_bar_gap(template_ppt, 30, dt_tbi, gap_tbi, 'Comforta')
ppt_tb.ppt_by_kota(template_ppt, 31, excel, 'indeks tbi kota', False)
ppt_tb.ppt_by_usiar(template_ppt, 32, excel, 'indeks tbi usiar', False)
ppt_tb.ppt_by_expandr(template_ppt, 33, excel, 'indeks tbi expandr', False)
ppt_tb.ppt_by_sex(template_ppt, 33, excel, 'indeks tbi sex', False)

ppt_tb.ppt_plot_bar_gap(template_ppt, 35, mind_share, gap_mind_share, 'Comforta')
ppt_tb.plot_nSampel_multirespon(template_ppt, 35, nSampel_mind_share, 'Tahoma', 10,
                            8, 1.25, 1.25, .25, -1, 0, multirespon_ = False)
ppt_tb.ppt_by_kota(template_ppt, 36, excel, 'mind_share_kota')
ppt_tb.ppt_by_usiar(template_ppt, 37, excel, 'mind_share_usiar')
ppt_tb.ppt_by_expandr(template_ppt, 38, excel, 'mind_share_expandr')
ppt_tb.ppt_by_sex(template_ppt, 38, excel, 'mind_share_sex')

ppt_tb.ppt_plot_bar_gap(template_ppt,39, mind_share_nonuser,gap_mind_share_nonuser, 'Comforta')
ppt_tb.plot_nSampel_multirespon(template_ppt, 39, nSampel_mind_share_nonUser, 'Tahoma', 10,
                            8, 1.25, 1.25, .25, -1, 0)
ppt_tb.ppt_by_kota(template_ppt, 40, excel, 'mind_share_kota_nonuser')
ppt_tb.ppt_by_usiar(template_ppt, 41, excel, 'mind_share_usiar_nonuser')
ppt_tb.ppt_by_expandr(template_ppt, 42, excel, 'mind_share_expandr_nonuser')
ppt_tb.ppt_by_sex(template_ppt, 42, excel, 'mind_share_sex_nonuser')

ppt_tb.ppt_plot_bar_gap(template_ppt, 43, dt_unaided, gap_unaided, 'Comforta')
ppt_tb.ppt_plot_bar_gap(template_ppt, 43, mind_share, None, None)
ppt_tb.plot_nSampel_multirespon(template_ppt, 43, nSampel_mult_unaided, 'Tahoma', 10,
                            7.5, 1.25, 2, .5, -1, 0, True, 0, 1)
ppt_tb.ppt_by_kota(template_ppt, 44, excel, 'unaided kota', True, True)
ppt_tb.ppt_by_usiar(template_ppt, 45, excel, 'unaided usiar', True, True)
ppt_tb.ppt_by_expandr(template_ppt, 46, excel, 'unaided expandr', True, True)
ppt_tb.ppt_by_sex(template_ppt, 46, excel, 'unaided sex', True, True)

ppt_tb.ppt_plot_bar_gap(template_ppt, 48, market_share, gap_market_share, 'Comforta')
ppt_tb.plot_nSampel_multirespon(template_ppt, 48, nSampel_market_share, 'Tahoma', 10,
                            8, 1.25, 1.25, .25, -1, 0, multirespon_ = False)
ppt_tb.ppt_by_kota(template_ppt, 49, excel, 'market_share_kota')
ppt_tb.ppt_by_usiar(template_ppt, 50, excel, 'market_share_usiar')
ppt_tb.ppt_by_expandr(template_ppt, 51, excel, 'market_share_expandr')
ppt_tb.ppt_by_sex(template_ppt, 51, excel, 'market_share_sex')

ppt_tb.ppt_plot_bar_gap(template_ppt, 53, commitment_share, gap_commitment_share)
ppt_tb.plot_nSampel_multirespon(template_ppt, 53, nSampel_market_share, 'Tahoma', 10,
                            8, 1.25, 1.25, .25, -1, 0, multirespon_ = False)
ppt_tb.ppt_by_kota(template_ppt, 54, excel, 'commitment_share_kota')
ppt_tb.ppt_by_usiar(template_ppt, 55, excel, 'commitment_share_usiar')
ppt_tb.ppt_by_expandr(template_ppt, 56, excel, 'commitment_share_expandr')
ppt_tb.ppt_by_sex(template_ppt, 56, excel, 'commitment_share_sex')

## Brand switching

In [33]:
ppt_tb.plot_brand_switching_bar(template_ppt, 59, brand_switch, n_sample)
ppt_tb.plot_bar_chart(template_ppt, 59, brand_switch['Brand'], brand_switch['Switching in'], 4.25, 1.675, 3.5, 3.75, 0, 250, [127, 127, 127])
ppt_tb.plot_bar_chart(template_ppt, 59, brand_switch['Brand'], brand_switch['Net switching'], 4, 1.675, 4.5, 3.75, -200, 200, [192, 0, 0])
ppt_tb.plot_bar_chart(template_ppt, 59, brand_switch['Brand'], brand_switch['Actual LU'], 7, 1.675, 2.5, 3.75, 0, 100, [64, 49, 82])
ppt_tb.plot_bar_chart(template_ppt, 59, brand_switch['Brand'], brand_switch['Prediksi LU'], 8.375, 1.675, 2.5, 3.75, 0, 100, [152, 72, 7])

ppt_tb.ppt_tabel_brand_switching(template_ppt, 60, tabel_brand_switching, 0.375, 2.125, 8.675, 3)

ppt_tb.plot_stacked_bar(template_ppt, 62,
                        list_data=[conv_rate['CR'], conv_rate['UOB'], conv_rate['ABNU']],
                        kategori=conv_rate['Brand'], kategori1=n_sample['n Sample'],
                        left=3.5, top=1.25, width=4, height=4)
ppt_tb.plot_bar_chart(template_ppt, 62, conv_rate['Brand'], conv_rate['TOM'], .75, 1.25, 5, 4, 0, 100,[25, 25, 77], n_sample['n Sample'], True)
ppt_tb.plot_bar_chart(template_ppt, 62, conv_rate['Brand'], conv_rate['NABU'], 7.75, 1.25, 4, 4, 0, 100,[192, 0, 0], n_sample['n Sample'])

## Brand Diagnostic & Competitor Landscape

In [34]:
ppt_tb.plot_tabel_brandDiac(template_ppt, 66, dt_diagnostic)
ppt_tb.plot_scatter_brandDiac(template_ppt, 67, dt_diagnostic, 6)

In [35]:
ppt_tb.ppt_competition_landscape(template_ppt, 71,excel, 'competitor landscape', 0.375, 1, 8.675, 3, 9, 9, .375, .375, 1.5)

ppt_tb.ppt_competition_landscape(template_ppt, 72, excel, 'competitor kotaJabodetabek', 2, 1.25, 7, 2.5, 8, 8, .25, .5, 1)
ppt_tb.ppt_competition_landscape(template_ppt, 72, excel, 'competitor kotaBandung', 2, 4.25, 7, 2.5, 8, 8, .25, .5, 1)

ppt_tb.ppt_competition_landscape(template_ppt, 73, excel, 'competitor kotaSemarang', 2, 1.25, 7, 2.5, 8, 8, .25, .5, 1)
ppt_tb.ppt_competition_landscape(template_ppt, 73, excel, 'competitor kotaYogyakarta', 2, 4.25, 7, 2.5, 8, 8, .25, .5, 1)

ppt_tb.ppt_competition_landscape(template_ppt, 74, excel, 'competitor kotaSurabaya', 2.5, .875, 6.5, 2, 7, 8, .15, .5, 1)
ppt_tb.ppt_competition_landscape(template_ppt, 74, excel, 'competitor kotaMalang', 2.5, 3, 6.5, 2, 7, 8, .15, .5, 1)
ppt_tb.ppt_competition_landscape(template_ppt, 74, excel, 'competitor kotaDenpasar', 2.5, 5.125, 6.5, 2, 7, 8, .15, .5, 1)

ppt_tb.ppt_competition_landscape(template_ppt, 75, excel, 'competitor kotaMedan', 2.5, .875, 6.5, 2, 7, 8, .15, .5, 1)
ppt_tb.ppt_competition_landscape(template_ppt, 75, excel, 'competitor kotaPalembang', 2.5, 3, 6.5, 2, 7, 8, .15, .5, 1)
ppt_tb.ppt_competition_landscape(template_ppt, 75, excel, 'competitor kotaPekanbaru', 2.5, 5.125, 6.5, 2, 7, 8, .15, .5, 1)

ppt_tb.ppt_competition_landscape(template_ppt, 76, excel, 'competitor kotaBanjarmasin', 2.5, .875, 6.5, 2, 7, 8, .15, .5, 1)
ppt_tb.ppt_competition_landscape(template_ppt, 76, excel, 'competitor kotaBalikpapan', 2.5, 3, 6.5, 2, 7, 8, .15, .5, 1)
ppt_tb.ppt_competition_landscape(template_ppt, 76, excel, 'competitor kotaSamarinda', 2.5, 5.125, 6.5, 2, 7, 8, .15, .5, 1)

ppt_tb.ppt_competition_landscape(template_ppt, 77, excel, 'competitor kotaMakassar', 2, 1.25, 7, 2.5, 8, 8, .25, .5, 1)
ppt_tb.ppt_competition_landscape(template_ppt, 77, excel, 'competitor kotaManado', 2, 4.25, 7, 2.5, 8, 8, .25, .5, 1)

## General Information

### Media Habit

In [36]:
ppt_tb.plot_bar_chart(template_ppt, 80, tab_mhabit['Kriteria'][:len(tab_mhabit)-1], 
               tab_mhabit['Presentase'][:len(tab_mhabit)-1],
               1, 1.75, 6, 4, 0, 100,[37, 64, 97], None, True, 12, 40)
ppt_tb.plot_nSampel_multirespon(template_ppt, 80, tab_mhabit, 'Tahoma', 10,
                            4.5, 5.5, 2, .5, -1, -2, True, -1, -1)

ppt_tb.plot_bar_chart(template_ppt, 81, tab_mhabit_tv['Kriteria'][:len(tab_mhabit_tv)-1], 
               tab_mhabit_tv['Presentase'][:len(tab_mhabit_tv)-1],
               6, 2, 2.25, 3.5, 0, 100,[37, 64, 97], None, True, 12, 40)
ppt_tb.plot_nSampel_multirespon(template_ppt, 81, tab_mhabit_tv, 'Tahoma', 10,
                            6.5, 5.5, 2, .5, -1, -2, True, -1, -1)

ppt_tb.plot_bar_chart(template_ppt, 82, tab_mhabit_acaratv['Kriteria'][:len(tab_mhabit_acaratv)-1], 
               tab_mhabit_acaratv['Presentase'][:len(tab_mhabit_acaratv)-1],
               .5, 1.5, 4, 4.25, 0, 100,[37, 64, 97], None, True, 12, 40)
ppt_tb.plot_nSampel_multirespon(template_ppt, 82, tab_mhabit_acaratv, 'Tahoma', 10,
                            4.5, 5.25, 2, .5, -1, -2, True, -1, -1)

ppt_tb.plot_bar_chart(template_ppt, 83, tab_mhabit_koran['Kriteria'][:len(tab_mhabit_koran)-1], 
               tab_mhabit_koran['Presentase'][:len(tab_mhabit_koran)-1],
               6, 2, 3, 3.25, 0, 100, [37, 64, 97],None, True, 12, 40)
ppt_tb.plot_nSampel_multirespon(template_ppt, 83, tab_mhabit_koran, 'Tahoma', 10,
                            6.5, 5.5, 2, .5, -1, -2, True, -1, -1)

ppt_tb.plot_bar_chart(template_ppt, 84, tab_mhabit_majalah['Kriteria'][:len(tab_mhabit_majalah)-1], 
               tab_mhabit_majalah['Presentase'][:len(tab_mhabit_majalah)-1],
               6, 2, 3, 3.25, 0, 100, [37, 64, 97],None, True, 12, 40)
ppt_tb.plot_nSampel_multirespon(template_ppt, 84, tab_mhabit_majalah, 'Tahoma', 10,
                            6.5, 5.5, 2, .5, -1, -2, True, -1, -1)

ppt_tb.plot_bar_chart(template_ppt, 85, tab_mhabit_tabloid['Kriteria'][:len(tab_mhabit_tabloid)-1], 
               tab_mhabit_tabloid['Presentase'][:len(tab_mhabit_tabloid)-1],
               6, 2, 2.25, 3.25, 0, 100, [37, 64, 97],None, True, 12, 40)
ppt_tb.plot_nSampel_multirespon(template_ppt, 85, tab_mhabit_tabloid, 'Tahoma', 10,
                            4.5, 5.25, 2, .5, -1, -2, True, -1, -1)

### Internet & Media Sosial

In [37]:
ppt_tb.pie_chart_olshop(template_ppt, 87, profil_olshop_persen)
ppt_tb.plot_data_olshop_kota(template_ppt, 87, profil_olshop_persen)
ppt_tb.plot_nSampel_multirespon(template_ppt, 87, profil_olshop_persen, 'Lato Light', 10,
                            .5, 4.5, 1, .25, -1, -1)

ppt_tb.plot_bar_chart(template_ppt, 88, tab_ism_jenis_barang['Kriteria'][:len(tab_ism_jenis_barang)-1], 
               tab_ism_jenis_barang['Presentase'][:len(tab_mhabit_tabloid)-1],
               4, 1.5, 4.25, 3.75, 0, 100, [37, 64, 97],None, True, 12, 40)
ppt_tb.plot_bar_chart(template_ppt, 88, tab_ism_pembayaran['Kriteria'][:len(tab_ism_pembayaran)-1], 
               tab_ism_pembayaran['Presentase'][:len(tab_ism_pembayaran)-1],
               .25, 4.375, 4.25, 2.665, 0, 100, [37, 64, 97],None, True, 12, 40)
ppt_tb.plot_nSampel_multirespon(template_ppt, 88, tab_ism_jenis_barang, 'Lato Light', 10,
                            7, 5.25, 2, .5, -1, -2, True, -1, -1)
ppt_tb.plot_text(template_ppt, 88, 'Rata-rata pengeluaran', rata_pengeluaran, 'Lato Light', 15,
                            .25, 2.75, 2, .5)

ppt_tb.plot_bar_chart(template_ppt, 90, tab_ism_gadget['Kriteria'][:len(tab_ism_gadget)-1], 
               tab_ism_gadget['Presentase_1'][:len(tab_ism_gadget)-1],
               .25, 1.75, 3.5, 3.5, 0, 100, [37, 64, 97],None, True, 12, 40)
ppt_tb.plot_bar_chart(template_ppt, 90, tab_ism_gadget['Kriteria'][:len(tab_ism_gadget)-1], 
               tab_ism_gadget['Presentase_2'][:len(tab_ism_gadget)-1],
               4.5, 1.75, 2.25, 3.5, 0, 100, [85, 142, 213],None, False, 12, 40)
ppt_tb.plot_nSampel_multirespon(template_ppt, 90, tab_ism_gadget, 'Lato Light', 10,
                            2, 5.25, 2, .5, -1, 1, True, -1, 2)

ppt_tb.plot_bar_chart(template_ppt, 91, tab_ism_chat['Kriteria'][:len(tab_ism_chat)-1], 
               tab_ism_chat['Presentase'][:len(tab_ism_chat)-1],
               1.25, 2.25, 3.75, 1.5, 0, 100, [37, 64, 97],None, True, 10, 40)

ppt_tb.plot_bar_chart(template_ppt, 92, tab_ism_sosmed['Kriteria'][:len(tab_ism_sosmed)-1], 
               tab_ism_sosmed['Presentase_1'][:len(tab_ism_sosmed)-1],
               1.5, 4.75, 2, 1.75, 0, 100, [37, 64, 97],None, True, 10, 40)
ppt_tb.plot_bar_chart(template_ppt, 92, tab_ism_sosmed['Kriteria'][:len(tab_ism_sosmed)-1], 
               tab_ism_sosmed['Presentase_2'][:len(tab_ism_sosmed)-1],
               3.5, 4.75, 1.5, 1.75, 0, 100, [85, 142, 213],None, False, 10, 40)

In [38]:
ppt_tb.plot_bar_chart(template_ppt, 89, tab_ism_aktivitas['Kriteria'][:len(tab_ism_aktivitas)-1], 
               tab_ism_aktivitas['Presentase'][:len(tab_ism_aktivitas)-1],
               3.5, 2, 3.5, 3.25, 0, 100, [37, 64, 97],None, True, 10, 40)
ppt_tb.plot_bar_chart(template_ppt, 89, tab_ism_waktu['Kriteria'][:len(tab_ism_waktu)-1], 
               tab_ism_waktu['Presentase'][:len(tab_ism_waktu)-1],
               7, 2, 3.5, 2.375, 0, 100, [37, 64, 97],None, True, 10, 40)
ppt_tb.plot_growing_text(template_ppt, 89, tab_ism_akses.iloc[:-1,:], [20, 15, 10], 'Lato Light',
                     .25, 2.25, 3, 1)
ppt_tb.plot_nSampel_multirespon(template_ppt, 89, tab_ism_aktivitas, 'Lato Light', 10,
                            .5, 5.75, 2, .5, -1, -2, True, -1, -1)

ppt_tb.plot_bar_chart(template_ppt, 91, tab_ism_email['Kriteria'][:len(tab_ism_email)-1], 
               tab_ism_email['Presentase'][:len(tab_ism_email)-1],
               1.5, 4.25, 3, 2.25, 0, 100, [37, 64, 97],None, True, 10, 40)
ppt_tb.plot_bar_chart(template_ppt, 91, tab_ism_media_upload['Kriteria'][:len(tab_ism_media_upload)-1], 
               tab_ism_media_upload['Presentase'][:len(tab_ism_media_upload)-1],
               4.5, 2.5, 3.5, 3.5, 0, 100, [37, 64, 97],None, True, 10, 40)
ppt_tb.plot_bar_chart(template_ppt, 91, tab_ism_media_download['Kriteria'][:len(tab_ism_media_download)-1], 
               tab_ism_media_download['Presentase'][:len(tab_ism_media_download)-1],
               7.5, 2.5, 3.25, 4, 0, 100, [37, 64, 97],None, True, 10, 40)

ppt_tb.plot_bar_chart(template_ppt, 92, tab_ism_game['Kriteria'][:len(tab_ism_game)-1], 
               tab_ism_game['Presentase'][:len(tab_ism_game)-1],
               .125, 1.5, 2.875, 2.75, 0, 100, [37, 64, 97],None, True, 10, 40)
ppt_tb.plot_bar_chart(template_ppt, 92, tab_ism_berita['Kriteria'][:len(tab_ism_berita)-1], 
               tab_ism_berita['Presentase'][:len(tab_ism_berita)-1],
               2.25, 1.5, 2.875, 2.75, 0, 100, [37, 64, 97],None, True, 10, 40)
ppt_tb.plot_bar_chart(template_ppt, 92, tab_ism_streaming['Kriteria'][:len(tab_ism_streaming)-1], 
               tab_ism_streaming['Presentase'][:len(tab_ism_streaming)-1],
               5.5, 2, 3.5, 3.5, 0, 100, [37, 64, 97],None, True, 10, 40)
ppt_tb.plot_bar_chart(template_ppt, 92, tab_ism_olshop['Kriteria'][:len(tab_ism_olshop)-1], 
               tab_ism_olshop['Presentase'][:len(tab_ism_olshop)-1],
               7.75, 2, 3.5, 4, 0, 100, [37, 64, 97],None, True, 10, 40)


## save

In [39]:
template_ppt.save('test laporan top brand2.pptx')